<a href="https://colab.research.google.com/github/BigFishDreamWater/-/blob/master/TFtestv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
 raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras import Model
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
!cat /proc/cpuinfo

HAHA 


In [0]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test=x_train/255.0,x_test/255.0

x_train=x_train[...,tf.newaxis]
x_test=x_test[...,tf.newaxis]

train_ds=tf.data.Dataset.from_tensor_slices(
    (x_train,y_train)).shuffle(10000).batch(32)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)



# 新段落

In [0]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1= Conv2D(32,3,activation='relu')
        self.flatten=Flatten()
        self.d1=Dense(128,activation='relu')
        self.d2=Dense(10)
    def call(self,x):
        x=self.conv1(x)
        x=self.flatten(x)
        x=self.d1(x)
        return self.d2(x)
model=MyModel()

In [0]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer=tf.keras.optimizers.Adam()

train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function

def train_step(images,labels):
    with tf.GradientTape()as tape:
        predcitions=model(images,training=True)
        loss=loss_object(labels,predcitions)
    gradients=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,predcitions)

In [0]:
@tf.function
def test_step(images,labels):

    predictions=model(images,training=False)
    t_loss=loss_object(labels,predictions)

    test_loss(t_loss)
    test_accuracy(labels,predictions)
EPOCHS=5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    for images,labels in train_ds:

        train_step(images,labels)
    
    for test_images,test_labels in test_ds:
        test_step(test_images,test_labels)
    template='Epoch{},Loss:{},Accuracy:{},Test Loss:{},Test Accuracy:{}' 
    print(template.format(epoch+1,train_loss.result(),train_accuracy.result()*100,test_loss.result(),test_accuracy.result()*100))